# [HW2] Topic Modeling
1. Crawling News
2. Preprocessing
3. Build Term-Document Matrix
4. Topic modeling
5. Visualization

```
🔥 이번 시간에는 Topic Modeling를 직접 크롤링한 뉴스 데이터에 대해서 수행해보는 시간을 갖겠습니다. 

먼저 네이버에서 뉴스 기사를 간단하게 크롤링합니다.
기본적인 전처리 이후 Term-document Matrix를 만들고 이를 non-negative factorization을 이용해 행렬 분해를 하여 Topic modeling을 수행합니다.

t-distributed stochastic neighbor embedding(T-SNE) 기법을 이용해 Topic별 시각화를 진행합니다.
```

In [32]:
!pip install newspaper3k;
!pip install konlpy;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
# 크롤링에 필요한 패키지 설치
from bs4 import BeautifulSoup
from newspaper import Article
from time import sleep
from time import time
from dateutil.relativedelta import relativedelta
from datetime import datetime
from multiprocessing import Pool
import json
import requests
import re
import sys

```
💡 Crawling(크롤링)이란?

크롤링은 웹 페이지에서 필요한 데이터를 추출해내는 작업을 말합니다.
이번 시간에는 정적 페이지인 네이버의 뉴스 신문 기사 웹페이지를 크롤링합니다.

HTML은 설명되어 있는 자료가 많기 때문에 생략하도록 하겠습니다.
HTML 구조 파악 및 태그에 대한 설명은 아래 참고자료를 살펴봐주세요 !
```

참고: [위키피디아: 정적페이지](https://ko.wikipedia.org/wiki/%EC%A0%95%EC%A0%81_%EC%9B%B9_%ED%8E%98%EC%9D%B4%EC%A7%80)

참고: [생활코딩: HTML](https://opentutorials.org/course/2039)

In [34]:
def crawl_news(query: str=None, crawl_num: int=1000, workers: int=4):
    '''뉴스 기사 텍스트가 담긴 list를 반환합니다.

    Keyword arguments:
    query -- 검색어 (default None)
    crawl_num -- 수집할 뉴스 기사의 개수 (defualt 1000)
    workers -- multi-processing시 사용할 thread의 개수 (default 4)
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
    articleList = []
    crawled_url = set()
    keyboard_interrupt = False
    t = time()
    idx = 0
    page = 1

    
    # 서버에 url 요청의 결과를 선언
    res = requests.get(url.format(query))
    sleep(0.5)
    # res를 parsing할 parser를 선언
    bs = BeautifulSoup(res.text, 'html.parser')
    
    with Pool(workers) as p:
        while idx < crawl_num:  #기사의 갯수.          
            table = bs.find('ul', {'class': 'list_news'}) #하위에 li태그 "bx" class로 뉴스거리가 나온다. 아래는 id로 구분하네...
            li_list = table.find_all('li', {'id': re.compile('sp_nws.*')}) #id로 구분 .줄바꿈 외 모든, *0번이상
            area_list = [li.find('div', {'class':'news_area'}) for li in li_list] #뉴스거리 중에서 하나 뽑아서 그중에 news_wrap api_ani_sent 를 클래스로 가지는 div 안으로 접근
            a_list = [area.find('a', {'class':'news_tit'}) for area in area_list] #a태그 찾고 저장.

            # 즉 각 뉴스거리중에서 a태그만 a_list에 저장한다.
            
            for n in a_list[:min(len(a_list), crawl_num-idx)]: #한 페이지당 미리 정해둔 크기보다 더 기사가 많으면 짜른다.
                articleList.append(n.get('title')) #a태그의 title을 가져온다. articleList에 저장.
                idx += 1                           
            page += 1                                #페이지 넘기기 위한 변수수

            pages = bs.find('div', {'class': 'sc_page_inner'}) #다음페이지로 넘기기 위한 div태그그
            next_page_url = [p for p in pages.find_all('a') if p.text == str(page)][0].get('href') #찾고자 하는 page넘버와 같은 숫자를 찾고 이것의 하이퍼링크를 저장합니다.

            req = requests.get('https://search.naver.com/search.naver' + next_page_url) 
            bs = BeautifulSoup(req.text, 'html.parser')
    return articleList

```
🔥 이제 '구글'이라는 이름으로 뉴스 기사 1000개의 제목을 크롤링하겠습니다.
```

In [35]:
query = '구글'

articleList = crawl_news(query)

In [36]:
articleList[:10]

['김정은 딸, 구글 北 관련 검색어 1위',
 '리설주 닮은 ‘김정은 딸’에 인터넷검색량 급증...北 관련 구글 검색 1위',
 '미래 사무 공간은?…구글 신사옥 공개',
 '美구글플레이 반독점 위반 소송, 원고 2천100만명 집단소송 비화',
 '‘대졸’은 선택?…구글·델타·IBM 등 대기업에 美 주정부들, 학력요건 완화',
 '"소비자 기만 허위 리뷰"...구글, 미국서 125억원 벌금',
 '머스크 "애플·구글서 트위터 제거하면, 스마트폰 생산할 것"',
 '구글 "연말까지 10억명의 \'지속가능한 행동\' 이끌 것"',
 '이노스, KU 라인업 구글 OS 출시 기념 브랜드위크 프로모션 진행',
 "네오위즈, 신작 '브라운더스트 스토리' 구글·애플 앱 마켓 정식 출시"]

```
🔥 태거(tagger)를 이용해 한글 명사와 알파벳만을 추출해서 term-document matrix (tdm)을 만들겠습니다.

태거(tagger)는 tokenization에서 조금 더 자세히 다루도록 하겠습니다.
```

참고: [konlpy: morph analyzer](https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/)

## Preprocessing

In [37]:
from konlpy.tag import Okt
from collections import Counter
import json

# Okt 형태소 분석기 선언
t = Okt()

words_list_ = []
vocab = Counter()
tag_set = set(['Noun', 'Alpha'])
stopwords = set(['글자'])

for i, article in enumerate(articleList): #기사의 제목 하나를 다루겠다. article은 title
    if i % 100 == 0:
        print(i)
    
    # tagger를 이용한 품사 태깅
    words = t.pos(article, norm=True, stem=True) #형태소 분석 후 형태소,품사 튜플 형태로 리스트 안에 들어감.
    ############################ ANSWER HERE ################################
    # TODO: 다음의 조건을 만족하는 단어의 리스트를 완성하세요.
    # 조건 1: 명사와 알파벳 tag를 가진 단어
    # 조건 2: 철자 길이가 2이상인 단어 
    # 조건 3: stopwords에 포함되지 않는 단어
    words = [w for w, t in words if t in tag_set and len(w) > 1 and w not in stopwords]
    '''
    words에서 하나씩 꺼내는데 품사가 noun이나 alpha 안에 속하고 두 글자 이상의
    단어이면서 '글자' 라는 단어를 포함하지 않는 words의 단어인 w를 리스트에 저장한다.
    '''
    #########################################################################        

    vocab.update(words) #words에서 중복된 단어를 세고 딕셔너리형태로 바꾸고 vocab에 저장.
    words_list_.append((words, article)) #하나의 기사에 대한 단어들과 그 기사의 제목을을 튜플 형태로 리스트 안에 저장한다.
    
vocab = sorted([w for w, freq in vocab.most_common(10000)])#최고 빈도를 보이는 값부터 10000개 중 단어를 vocab에 저장하고 이를 사전순으로 정렬합니다.

word2id = {w: i for i, w in enumerate(vocab)} #단어마다 인덱스를 부여합니다. 여기서 단어는 유일합니다.
words_list = [] 
for words, article in words_list_: 
    words = [w for w in words if w in word2id] #하나의 기사에 대한 단어들 중 인덱스로 나타낼 수 있는 단어만 저장한다. 인덱스로 나타낼 수 없으면 드롭
    if len(words) > 10: #한 단어의 글자수가 10이 넘으면
        words_list.append((words, article)) #리스트에 그 단어들과 기사제목을 튜플로 묶어서 리스트에 저장.
        
del words_list_#전처리가 끝났기 때문에 바이바이

0
100
200
300
400
500
600
700
800
900


In [38]:
len(vocab)

1667

## Build document-term matrix

```
🔥 이제 document-term matrix를 만들어보겠습니다.
document-term matrix는 (문서 개수 x 단어 개수)의 Matrix입니다.
```

참고: [Document-Term Matrix](https://wikidocs.net/24559)

In [39]:
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

dtm = np.zeros((len(words_list), len(vocab)), dtype=np.float32) #기사갯수(crawl_num : 1000) 행 , 단어갯수(10000) 열.???
for i, (words, article) in enumerate(words_list): #word_list는 한 아이템이 words와 article로 되어 있으며 article은 기사의 제목을 words는 인덱스로서 표현현 수 있는 단어의 리스트가 있다.
    for word in words: #단어를 꺼내서
        dtm[i, word2id[word]] += 1 #한 기사에서 반복문으로 꺼낸 단어의 인덱스에 해당하는 장소에 1을 더한다. 즉 빈도수를 구한다.
        
dtm = TfidfTransformer().fit_transform(dtm) 
print(dtm.shape)

(88, 1667)


```
🔥 document-term matrix를 non-negative factorization(NMF)을 이용해 행렬 분해를 해보겠습니다.

💡 Non-negative Factorization이란?

NMF는 주어진 행렬 non-negative matrix X를 non-negative matrix W와 H로 행렬 분해하는 알고리즘입니다.
이어지는 코드를 통해 W와 H의 의미에 대해 파악해봅시다.
```
참고: [Non-negative Matrix Factorization](https://angeloyeo.github.io/2020/10/15/NMF.html)

## Topic modeling

In [40]:
# Non-negative Matrix Factorization
from sklearn.decomposition import NMF

K=5
nmf = NMF(n_components=K, alpha=0.1)

```
🔥 sklearn의 NMF를 이용해 W와 H matrix를 구해봅시다.
W는 document length x K, H는 K x term length의 차원을 갖고 있습니다.
W의 하나의 row는 각각의 feature에 얼만큼의 가중치를 줄 지에 대한 weight입니다.
H의 하나의 row는 하나의 feature를 나타냅니다.

우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [41]:
W = nmf.fit_transform(dtm)
H = nmf.components_

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [42]:
print(W.shape, H.shape) # 5 topics 91 articles 1662 words

(88, 5) (5, 1667)


In [43]:
print(W[:10])
print(H)

[[0.00373511 0.00563938 0.00494793 0.02789384 0.00838112]
 [0.00330832 0.00597298 0.00363424 0.05958978 0.00517269]
 [0.00449182 0.00938229 0.00468577 0.04797763 0.0069283 ]
 [0.0065058  0.01053751 0.01308415 0.07124657 0.00603484]
 [0.00369845 0.01600544 0.00630203 0.03543119 0.00910201]
 [0.         0.         0.01907239 0.25615781 0.        ]
 [0.68542502 0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.72264444]
 [0.0065058  0.01053751 0.01308415 0.07124657 0.00603484]
 [0.00100092 0.00121786 0.         0.11873454 0.07613949]]
[[0.00131837 0.         0.         ... 0.         0.         0.01290952]
 [0.00249198 0.         0.         ... 0.         0.         0.01911161]
 [0.00086241 0.         0.         ... 0.         0.         0.00094993]
 [0.04631066 0.         0.         ... 0.         0.         0.00491131]
 [0.00416121 0.         0.         ... 0.         0.         0.00087907]]


```
🔥 우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [44]:
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]: #k번째 토픽에서 가장 오름차순으로 정렬 -> 뒤부터니 큰 것부터 20개의 인덱스만 추출출
        print(vocab[index], end=' ') #빈도수가 가장 높은 것부터 해당하는 인덱스를 넣어서 프린트.
    print()

0th topic
순위 아이 달성 톡시 RPG 전신 신규 매출 인기 구글 니케 승리 여신 시프트 시장 기록 게임 세계관 미도 무기 
1th topic
컴투스 플레이 올해 후보 게임 선정 인기 홀딩스 워킹데드 스타즈 구글 크로니클 컴프 우승 프로야구 서머 공개 음악 승리 여신 
2th topic
사업 광고 트립 마이 리얼 조현아 총괄 트위터 숙박 전무 예정 프로그램 구글 인사 투자 서비스 프랑스 차량 설법 개발 
3th topic
팀뷰어 클라우드 플랫폼 산업 레이스 AR 프론트라 마켓 구글 지원 예약 로컬 테이블 레드 서비스 식당 개최 위메프 인프라 전국 
4th topic
퀴즈 로이 본사 스킴 신입 조세호 비하인드 버킷 감탄 시작 린지 아이스 유재석 종합 구글 이세돌 홍보 대국 담당 신입사원 


```
🔥 이번에는 W에서 하나의 Topic (W의 n번째 column)에 접근해서 해당 topic에 대해 값이 가장 높은 3개의 뉴스 기사 제목을 출력해보겠습니다.
```

In [45]:
for k in range(K):
    print(f"==={k}th topic===")
    for index in W[:, k].argsort()[::-1][:3]: #각 topic에 따른 가장 높은 기사들 3개. 가장높은?
        print(words_list[index][1]) #두번째 값이 제목목
    print('\n')

===0th topic===
아이톡시 RPG ‘전신’, 구글 신규 매출 순위 12위, 인기 순위 1위 달성
아이톡시 RPG '전신' 구글 신규 매출 순위 12위와 인기 순위 1위 달성
아이톡시 RPG '전신' 구글 신규 매출 순위 12위와 인기 순위 1위 달성


===1th topic===
컴투스홀딩스 '워킹데드: 올스타즈', 구글 플레이 '올해를 빛낸 인기 게임' 후보 선정
컴투스홀딩스 '워킹데드: 올스타즈', 구글 플레이 '올해를 빛낸 인기 게임' 후보 선정
컴투스홀딩스 '워킹데드: 올스타즈', 구글 플레이 '올해를 빛낸 인기 게임' 후보 선정


===2th topic===
마이리얼트립, 구글・트위터 광고사업 총괄 조현아 전무 영입…숙박사업 새로운 프로그램 선보일 예정
마이리얼트립, 구글・트위터 광고사업 총괄 조현아 전무 영입…숙박사업 새로운 프로그램 선보일 예정
[인사] 마이리얼트립, 구글・트위터 광고사업 총괄 조현아 전무 영입


===3th topic===
팀뷰어, 구글 클라우드 마켓플레이스에 산업용 AR 플랫폼 '팀뷰어 프론트라인’ 지원
팀뷰어, 구글 클라우드 마켓플레이스에 산업용 AR 플랫폼 '팀뷰어 프론트라인’ 지원
팀뷰어, 구글 클라우드 마켓플레이스에 산업용 AR 플랫폼 '팀뷰어 프론트라인’ 등록


===4th topic===
[종합] '유퀴즈' 유재석X조세호, 52세 구글 본사 신입 로이스킴 비하인드 '감탄'…션이 시작한 '아이스버킷 챌린지'
[종합] '유퀴즈' 유재석X조세호, 52세 구글 본사 신입 로이스킴 비하인드 '감탄'…션이 시작한 '아이스버킷 챌린지'
[종합] '유퀴즈' 유재석X조세호, 52세 구글 본사 신입 로이스킴 비하인드 '감탄'…션이 시작한 '아이스버킷 챌린지'




```
❓ 2번째 토픽에 대해 가장 높은 가중치를 갖는 제목 5개를 출력해볼까요?
```

In [46]:
#TODO
index_list = W[:, 2].argsort()[::-1][:5] #1th topic , 정렬해서 뒤부터 다섯개 인덱스를를
for index in index_list:  #words_list에 넣어서 프린트
    print(words_list[index][1])

마이리얼트립, 구글・트위터 광고사업 총괄 조현아 전무 영입…숙박사업 새로운 프로그램 선보일 예정
마이리얼트립, 구글・트위터 광고사업 총괄 조현아 전무 영입…숙박사업 새로운 프로그램 선보일 예정
[인사] 마이리얼트립, 구글・트위터 광고사업 총괄 조현아 전무 영입
마이리얼트립, 조현아 구글 광고 사업 총괄 영입…숙박 서비스에 투자
프랑스 르노, 구글과 차량용 소프트웨어 개발. EV사업 신설법인에 퀄컴 출자


```
🔥 이번에는 t-SNE를 이용해 Topic별 시각화를 진행해보겠습니다.

💡 t-SNE는 무엇인가요?

t-Stochastic Neighbor Embedding(t-SNE)은 고차원의 벡터를 
저차원(2~3차원) 벡터로 데이터간 구조적 특징을 유지하며 축소를 하는 방법 중 하나입니다.

주로 고차원 데이터의 시각화를 위해 사용됩니다.
```

참고: [lovit: t-SNE](https://lovit.github.io/nlp/representation/2018/09/28/tsne/#:~:text=t%2DSNE%20%EB%8A%94%20%EA%B3%A0%EC%B0%A8%EC%9B%90%EC%9D%98,%EC%9D%98%20%EC%A7%80%EB%8F%84%EB%A1%9C%20%ED%91%9C%ED%98%84%ED%95%A9%EB%8B%88%EB%8B%A4.)

참고: [ratsgo: t-SNE](https://ratsgo.github.io/machine%20learning/2017/04/28/tSNE/)

## Visualization

In [47]:
from sklearn.manifold import TSNE

# n_components = 차원 수
tsne = TSNE(n_components=2, init='pca', verbose=1)

# W matrix에 대해 t-sne를 수행합니다.
W2d = tsne.fit_transform(W)

# 각 뉴스 기사 제목마다 가중치가 가장 높은 topic을 저장합니다.
topicIndex = [v.argmax() for v in W]

[t-SNE] Computing 87 nearest neighbors...
[t-SNE] Indexed 88 samples in 0.000s...
[t-SNE] Computed neighbors for 88 samples in 0.009s...
[t-SNE] Computed conditional probabilities for sample 88 / 88
[t-SNE] Mean sigma: 0.056874


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:986: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  FutureWarning,


[t-SNE] KL divergence after 250 iterations with early exaggeration: 50.207161
[t-SNE] KL divergence after 1000 iterations: -0.527971


In [48]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article for words, article in words_list],
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color'
)

# interaction
p.legend.location = "top_left"
hover = p.select({'type': HoverTool})
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)